# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifycdb
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifycdb')
except Exception as e:
    print(e)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [34]:
# Created Table such that the session id and itemInSession are the partition keys since we are retrieving data by userid and sessionid

first = 'CREATE TABLE IF NOT EXISTS song_heard_item (sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))'

try:
    session.execute(first)
except Exception as e:
    print(e)


                    

In [35]:
# Inserting Values to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_heard_item(sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [36]:
# The Artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = 'SELECT artist,song,length from song_heard_item WHERE sessionId = 338 AND itemInSession = 4'
result1 = session.execute(query)

for result in result1:
    print(result)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [24]:
# Created Table such that the table is created with an order, hence you can sort it on iteminsession. The 2 partition keys are created since since we are retrieving data by userid and sessionid
second = 'CREATE TABLE IF NOT EXISTS  song_playlist_session (userId int,sessionId int,itemInSession int,artist text, song text,firstName text, lastName text,PRIMARY KEY(userId, sessionId,itemInSession)) WITH CLUSTERING ORDER BY(sessionId ASC, itemInSession ASC)'

try:
    session.execute(second)
except Exception as e:
    print(e)


                    

In [25]:
# Inserting Values to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO  song_playlist_session (userId,sessionId,itemInSession,artist,song,firstName,lastName ) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [30]:
# The name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = 'SELECT artist, song, firstName, lastName from song_playlist_session WHERE userId =10 AND sessionId =182'
result2 = session.execute(query)

for result in result2 :
    print(result)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [31]:
# Created Table such that the partition keys uniquely identfy the songs as per the user id. The user id is the element that uniquely identifies each person 
third = 'CREATE TABLE IF NOT EXISTS user_retrieve (song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))'

try:
    session.execute(third)
except Exception as e:
    print(e)

                    

In [32]:
# Inserting Values to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_retrieve (song, userId, firstName, lastName) "
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))      

In [33]:
# The user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT userId,firstName,lastName from user_retrieve where song='All Hands Against His Own' " 

result3 = session.execute(query)

for result in result3:
    print(result)

Row(userid=29, firstname='Jacqueline', lastname='Lynch')
Row(userid=80, firstname='Tegan', lastname='Levine')
Row(userid=95, firstname='Sara', lastname='Johnson')


In [19]:
# Dropping Tables
query1 = 'DROP TABLE user1';
query2 = 'DROP TABLE artist1';
query3 = 'DROP TABLE song1';

query_list = [query1, query2, query3]

for query in query_list:
    session.execute(query)

In [20]:
# Closing Session
session.shutdown()
cluster.shutdown()